In [1]:
import torchtext
import re
import regex
import numpy as np
import random
from torchtext.data import get_tokenizer

In [2]:
en_alphabets= r"(?:(?![ა-ჰ])\p{L})" # this actually includes all the other languages too. only works with regex not re
alphabets= "([ა-ჰ])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = r"(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = r"(\w[.]\w[.](?:\w[.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "(\d)"
hex_encoded_chars = r"\\x.." # \x41
url_encoded_chars = r"\&\w+\;" # &amp;

geo_bc = "ძვ[.]წ[.]"
url_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"

# <stop> will be an actual sentence splitting token
# <sep> will just be a dot
def split_into_sentences_and_remove_en(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    # remove patterns which will be hard to remove after all english is removed
    text = re.sub(prefixes,"",text) # prefix
    text = re.sub(url_pattern,"",text) # full url
    text = re.sub(websites,"",text) # .tld
    text = re.sub(hex_encoded_chars,"",text) # \x41
    text = re.sub(url_encoded_chars,"",text) # &amp;
    if "Ph.D" in text: text = text.replace("Ph.D.","")
    
    text = regex.sub("\s" + en_alphabets + "[.] "," ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = regex.sub(en_alphabets + "[.]" + en_alphabets + "[.]" + en_alphabets + "[.]"," ",text)
    text = regex.sub(en_alphabets + "[.]" + en_alphabets + "[.]"," ",text)
    text = re.sub(acronyms+" "+starters," ",text)
    text = re.sub(" "+suffixes+"[.] "+starters," ",text)
    text = re.sub(" "+suffixes+"[.]"," ",text)
    text = regex.sub(en_alphabets,"",text)
    
    # eng removing done
    
    # dates/times
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(r"(\d+)\.(\d+)\.(\d+)",r"\1<prd>\2<prd>\3",text) # dates
    text = re.sub(geo_bc,"ძვ<prd>წ<prd>",text)
    
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "!.." in text: text = text.replace("!..","!<prd><prd>")
    
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text) # თ. თოფურია
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = text.replace("„","\"")
    text = text.replace("”","\"")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    fake_dot = "․"
    text = text.replace("<prd>",fake_dot) # FAKE DOT THIS IS NOT ACTUAL DOT
    assert fake_dot != "."
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [3]:
split_into_sentences_and_remove_en('20.20.2020.  დღეს არის 2020 წელი ძვ.წ. ჰმჰმ.')

['20․20․2020.', 'დღეს არის 2020 წელი ძვ․წ․ ჰმჰმ.']

In [4]:
split_into_sentences_and_remove_en('20.20.2020.  დღეს არის 2020 წელი ძვ.წ. ჰმჰმ.')

['20․20․2020.', 'დღეს არის 2020 წელი ძვ․წ․ ჰმჰმ.']

In [6]:
split_into_sentences_and_remove_en('შ.პ.ს. მაგარი რამეა.')

['შ․პ․ს․ მაგარი რამეა.']

In [7]:
split_into_sentences_and_remove_en('hi this is so cool.')

['.']

In [9]:
p_train = 0.7
p_test = 0.2
p_valid = 0.1
total_sentences = 0

with open('./no_en_data/ka_nse_train.txt', 'w') as f_train:
    with open('./no_en_data/ka_nse_test.txt', 'w') as f_test:
        with open('./no_en_data/ka_nse_valid.txt', 'w') as f_valid:
            def process_data_file(path: str):
                global total_sentences
                num_docs = 0
                with open(path, 'r') as input_file:
                    cur_doc = ''
                    for line in input_file:
                        cur_doc += line
                        if line.strip() == '': # assumption: document ends on an empty line
                            doc_sentences = split_into_sentences_and_remove_en(cur_doc)
                            if len(doc_sentences) == 0:
                                continue

                            r = random.random()
                            if r < p_train:
                                f = f_train
                            elif r < p_train+p_test:
                                f = f_test
                            else:
                                f = f_valid
                            for s in doc_sentences:
                                if len(s)==1:
                                    continue
                                f.write(s)
                                total_sentences+=1
                                f.write('\n')
                            f.write('\n')
                            
                            if total_sentences >= 1_000_000:
                                break
                            cur_doc = ''
                            num_docs+=1    
                            if num_docs%10000==0:
                                print(num_docs)
            process_data_file('./data/უცხოელი_მწერლები_234.txt')
            process_data_file('./data/ka.txt')
import os
for tp in ['train', 'test', 'valid']:
    os.system(f"sed -i -e '/./b' -e :n -e 'N;s/\\n$//;tn' ./no_en_data/ka_nse_{tp}.txt")
    # remove last line
    os.system(f"sed -i '$ d' ./no_en_data/ka_nse_{tp}.txt")

10000
10000
20000
30000
40000
50000


In [10]:
with open('./no_en_data/ka_nse_complete.txt', 'w') as f_train:
    def process_data_file(path: str):
        global total_sentences
        num_docs = 0
        with open(path, 'r') as input_file:
            cur_doc = ''
            for line in input_file:
                cur_doc += line
                if line.strip() == '': # assumption: document ends on an empty line
                    doc_sentences = split_into_sentences_and_remove_en(cur_doc)
                    if len(doc_sentences) == 0:
                        continue

                    f = f_train
                    for s in doc_sentences:
                        if len(s)==1:
                            continue
                        f.write(s)
                        f.write('\n')
                    f.write('\n')

                    cur_doc = ''
                    num_docs+=1    
                    if num_docs%10000==0:
                        print(num_docs)
    process_data_file('./data/უცხოელი_მწერლები_234.txt')
    process_data_file('./data/ka.txt')
import os
#for tp in ['train', 'test', 'valid']:
os.system(f"sed -i -e '/./b' -e :n -e 'N;s/\\n$//;tn' ./no_en_data/ka_nse_complete.txt")
# remove last line
os.system(f"sed -i '$ d' ./no_en_data/ka_nse_complete.txt")

10000
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
138000

2